In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from vamos import (
    # Tuning utilities
    ParamSpace, Real, Int, Categorical,
    RandomSearchTuner, TuningTask, EvalContext, Instance,
    # Config space builders
    build_nsgaii_config_space, config_from_assignment,
    # Optimization
    optimize, OptimizeConfig, make_problem_selection,
)
from vamos.foundation.metrics import compute_hypervolume

plt.style.use("ggplot")
print("Tuning utilities loaded successfully!")

## 1. Define a Custom Parameter Space

Create a search space with different parameter types:

In [ ]:
# Define a custom parameter space for NSGA-II
custom_space = ParamSpace(params={
    "pop_size": Int("pop_size", 20, 100),
    "crossover_prob": Real("crossover_prob", 0.6, 0.99),
    "crossover_eta": Real("crossover_eta", 5.0, 30.0),
    "mutation_eta": Real("mutation_eta", 5.0, 30.0),
})

# Sample a few configurations to see what they look like
rng = np.random.default_rng(42)
for i in range(3):
    sample = custom_space.sample(rng)
    print(f"Sample {i+1}: {sample}")

## 2. Simple Random Search Tuning

Use `RandomSearchTuner` to find good hyperparameters for NSGA-II on ZDT1:

In [ ]:
# Create problem selection
selection = make_problem_selection("zdt1", n_var=10)
problem = selection.instantiate()

# Reference point for hypervolume (slightly worse than nadir)
ref_point = np.array([1.1, 1.1])

# Define evaluation function: run NSGA-II and return hypervolume
def evaluate_config(config: dict, ctx: EvalContext) -> float:
    """Evaluate a configuration and return hypervolume (higher = better)."""
    from vamos import NSGAIIConfig
    
    # Build algorithm config from sampled parameters
    algo_config = (
        NSGAIIConfig()
        .pop_size(int(config["pop_size"]))
        .crossover("sbx", prob=config["crossover_prob"], eta=config["crossover_eta"])
        .mutation("pm", prob=0.1, eta=config["mutation_eta"])
        .engine("numpy")
        .fixed()
    )
    
    # Run optimization with small budget
    opt_config = OptimizeConfig(
        problem=problem,
        algorithm="nsgaii",
        algorithm_config=algo_config,
        termination=("n_eval", ctx.budget),
        seed=ctx.seed,
    )
    result = optimize(opt_config)
    
    # Compute hypervolume
    hv = compute_hypervolume(result.F, ref_point)
    return hv

In [ ]:
# Create tuning task
task = TuningTask(
    name="nsgaii_zdt1_tuning",
    param_space=custom_space,
    instances=[Instance(name="zdt1", n_var=10)],
    seeds=[42],  # Single seed for speed
    budget_per_run=500,  # Small budget per evaluation
    maximize=True,  # Maximize hypervolume
)

# Run random search with 10 trials
tuner = RandomSearchTuner(task=task, max_trials=10, seed=123)
best_config, history = tuner.run(evaluate_config, verbose=True)

print(f"\nBest configuration found:")
for k, v in best_config.items():
    print(f"  {k}: {v:.4f}" if isinstance(v, float) else f"  {k}: {v}")

In [ ]:
# Visualize tuning progress
scores = [trial.score for trial in history]
best_so_far = np.maximum.accumulate(scores)

fig, ax = plt.subplots(figsize=(8, 4))
ax.plot(scores, 'o-', alpha=0.6, label="Trial score")
ax.plot(best_so_far, 'r-', linewidth=2, label="Best so far")
ax.set_xlabel("Trial")
ax.set_ylabel("Hypervolume")
ax.set_title("Random Search Tuning Progress")
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 3. Using Built-in Config Spaces

VAMOS provides pre-built configuration spaces for all algorithms:

In [ ]:
# Get the built-in NSGA-II config space
nsgaii_space = build_nsgaii_config_space()

print(f"Algorithm: {nsgaii_space.algorithm_name}")
print(f"\nParameters ({len(nsgaii_space.params)}):")
for param in nsgaii_space.params:
    print(f"  - {param.name}: {type(param).__name__}")

In [ ]:
# Sample from the built-in space
rng = np.random.default_rng(42)
sampled = nsgaii_space.sample(rng)

print("Sampled configuration:")
for k, v in sampled.items():
    print(f"  {k}: {v}")

In [ ]:
# Convert sampled assignment to a concrete config object
concrete_config = config_from_assignment("nsgaii", sampled)

print(f"Pop size: {concrete_config.pop_size}")
print(f"Crossover: {concrete_config.crossover}")
print(f"Mutation: {concrete_config.mutation}")

## 4. Multi-Instance Tuning

Tune across multiple problem instances for more robust configurations:

In [ ]:
# Define multiple instances
instances = [
    Instance(name="zdt1_small", n_var=10),
    Instance(name="zdt1_medium", n_var=20),
]

# Evaluation function that handles different instances
def evaluate_multi_instance(config: dict, ctx: EvalContext) -> float:
    from vamos import NSGAIIConfig
    
    # Create problem based on instance
    prob = make_problem_selection("zdt1", n_var=ctx.instance.n_var).instantiate()
    
    algo_config = (
        NSGAIIConfig()
        .pop_size(int(config["pop_size"]))
        .crossover("sbx", prob=config["crossover_prob"], eta=config["crossover_eta"])
        .mutation("pm", prob=0.1, eta=config["mutation_eta"])
        .engine("numpy")
        .fixed()
    )
    
    result = optimize(OptimizeConfig(
        problem=prob,
        algorithm="nsgaii",
        algorithm_config=algo_config,
        termination=("n_eval", ctx.budget),
        seed=ctx.seed,
    ))
    
    return compute_hypervolume(result.F, np.array([1.1, 1.1]))

In [ ]:
# Multi-instance tuning task
multi_task = TuningTask(
    name="nsgaii_multi_instance",
    param_space=custom_space,
    instances=instances,
    seeds=[42, 123],  # Multiple seeds
    budget_per_run=300,
    maximize=True,
    aggregator=np.mean,  # Average HV across instances and seeds
)

print(f"Task: {multi_task.name}")
print(f"Instances: {len(multi_task.instances)}")
print(f"Seeds: {len(multi_task.seeds)}")
print(f"Evaluations per config: {len(multi_task.instances) * len(multi_task.seeds)}")

In [ ]:
# Run tuning (fewer trials since each trial is more expensive)
multi_tuner = RandomSearchTuner(task=multi_task, max_trials=5, seed=456)
best_multi, history_multi = multi_tuner.run(evaluate_multi_instance, verbose=True)

print(f"\nBest robust configuration:")
for k, v in best_multi.items():
    print(f"  {k}: {v:.4f}" if isinstance(v, float) else f"  {k}: {v}")

## 5. Validate the Tuned Configuration

Compare tuned vs default configuration on a fresh run:

In [ ]:
from vamos import NSGAIIConfig

# Default configuration
default_cfg = NSGAIIConfig().pop_size(50).crossover("sbx", prob=0.9, eta=20).mutation("pm", prob=0.1, eta=20).fixed()

# Tuned configuration
tuned_cfg = (
    NSGAIIConfig()
    .pop_size(int(best_config["pop_size"]))
    .crossover("sbx", prob=best_config["crossover_prob"], eta=best_config["crossover_eta"])
    .mutation("pm", prob=0.1, eta=best_config["mutation_eta"])
    .fixed()
)

# Run both with same seed
test_problem = make_problem_selection("zdt1", n_var=20).instantiate()

result_default = optimize(OptimizeConfig(
    problem=test_problem, algorithm="nsgaii", algorithm_config=default_cfg,
    termination=("n_eval", 2000), seed=999
))

result_tuned = optimize(OptimizeConfig(
    problem=test_problem, algorithm="nsgaii", algorithm_config=tuned_cfg,
    termination=("n_eval", 2000), seed=999
))

hv_default = compute_hypervolume(result_default.F, ref_point)
hv_tuned = compute_hypervolume(result_tuned.F, ref_point)

print(f"Default config HV: {hv_default:.6f}")
print(f"Tuned config HV:   {hv_tuned:.6f}")
print(f"Improvement: {(hv_tuned - hv_default) / hv_default * 100:.2f}%")

In [ ]:
# Visual comparison
fig, ax = plt.subplots(figsize=(8, 6))

ax.scatter(result_default.F[:, 0], result_default.F[:, 1], 
           s=40, alpha=0.6, label=f"Default (HV={hv_default:.4f})")
ax.scatter(result_tuned.F[:, 0], result_tuned.F[:, 1], 
           s=40, alpha=0.6, label=f"Tuned (HV={hv_tuned:.4f})")

ax.set_xlabel("f1")
ax.set_ylabel("f2")
ax.set_title("Default vs Tuned NSGA-II on ZDT1")
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Summary

VAMOS provides flexible hyperparameter tuning:

| Component | Purpose |
|-----------|----------|
| `ParamSpace` | Define search space with Real, Int, Categorical params |
| `TuningTask` | Specify instances, seeds, budget, aggregation |
| `RandomSearchTuner` | Simple random search |
| `RacingTuner` | Efficient irace-style elimination |
| `build_*_config_space()` | Pre-built spaces for each algorithm |
| `config_from_assignment()` | Convert samples to concrete configs |

**Tips:**
- Start with `RandomSearchTuner` for quick exploration
- Use `RacingTuner` for larger budgets (more efficient)
- Tune across multiple instances for robust configs
- Use multiple seeds to reduce noise